In [1]:
%pylab inline
import sys
sys.path.append('../')

from managers.dataManager import DataManager as dm
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

import plotly.graph_objects as go
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataManager = dm(True)

In [3]:
data, index = dataManager.all_data_future_lag()

In [4]:
scaler = MinMaxScaler()

num_var = ['AREA','ALTO','DESCUENTO(%)','PRECIO']
for i in range(1,12):
    num_var.append('DIFF_{}'.format(i))
num_var.append('CANTIDAD')
x_num = data[num_var[:-1]].astype('float')
x_num_norm = scaler.fit_transform(x_num)

cat_var=[
    'MES','TIENDA','PUESTOS','COLOR_POS', 'SUBCATEGORIA_POS',
    'MATERIAL_POS','F_COVID','ACABADO','CATEGORIA','ORIGEN'
]
x_cat = data[cat_var].astype('category')
dummies = OneHotEncoder()
dummies.fit(x_cat)
x_cat_dummies = dummies.transform(x_cat).toarray()

x = np.append(x_num_norm,x_cat_dummies,axis=1)
y = data['CANTIDAD']

x_past = x[:-index]
x_train = x_past[:-int(len(x_past)*0.25)]
x_fut = x[-index:]

y_past = y[:-index]
y_train = y_past[:-int(len(y_past)*0.25)]

In [5]:
model = GridSearchCV(Lasso(), param_grid={'alpha': np.logspace(-3, 3, 10)},scoring='neg_mean_squared_error') #is better for small datasets: use if split is set to 2 or 3
model.fit(x_train,y_train)
params = model.best_params_
model_f = Lasso(**params)
model_f = model_f.fit(x_past,y_past)

In [49]:
pred_past = model.predict(x_past)

fut = data[-index:]
meses = fut.MES.unique()

for i in meses:
    mes = fut[fut['MES']==i].copy()
    
    x_num = mes[num_var[:-1]].astype('float')
    x_num_norm = scaler.transform(x_num)
    
    x_cat = mes[cat_var].astype('category')
    x_cat_dummies = dummies.transform(x_cat).toarray()
    
    x = np.append(x_num_norm,x_cat_dummies,axis=1)

    pred = model_f.predict(x)

    for j in meses:
        if j-i == 0:
            fut.loc[fut['MES']==j,'CANTIDAD'] = pred
        elif j-i > 0:
            fut.loc[fut['MES']==j,'CANTIDAD_{}'.format(j-i)] = pred
            fut.loc[fut['MES']==j,'DIFF_{}'.format(j-i)] = fut.loc[fut['MES']==j,'CANTIDAD_{}'.format(j-i)] - fut.loc[fut['MES']==j,'CANTIDAD_{}'.format(j-i+1)]

pred_fut = fut['CANTIDAD']
predicted = np.append(pred_past,pred_fut,axis=0)

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [10]:
pred_past = model.predict(x_past)
pred_fut = model_f.predict(x_fut)
predicted = np.append(pred_past,pred_fut,axis=0)

In [11]:
data['PREDICCION']= predicted
past = data[:-index]
fut = data[-index:]

In [12]:
d_futuro = fut.groupby(['DATE']).sum().reset_index()
d_pasado = past.groupby(['DATE']).sum().reset_index()
fig = go.Figure()
fig.add_scatter(x=d_pasado['DATE'], y=d_pasado['CANTIDAD'], mode='lines', name='Valores reales')
fig.add_scatter(x=d_pasado['DATE'], y=d_pasado['PREDICCION'], mode='lines', name='Valores predichos')
fig.add_scatter(x=d_futuro['DATE'], y=d_futuro['PREDICCION'], mode='lines', name='Valores predichos')
#fig.add_scatter(x=defi['DATE'], y=defi['CANTIDAD'], mode='lines', name='Valores reales')
#fig.add_scatter(x=defi['DATE'], y=defi['PREDICCION'], mode='lines', name='Valores predichos')